In [36]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [908 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18

In [41]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the Chrome driver
driver = webdriver.Chrome(options=chrome_options)

# Define the URL
url = "https://www.cbre.co.uk/property-search/office-space/listings/results?aspects=isLetting"

# Open the URL
driver.get(url)

# Print the page source to debug
print(driver.page_source)

# Wait for the properties to load
wait = WebDriverWait(driver, 30)  # Increase timeout to 30 seconds
try:
    properties = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "property-card")]')))
except Exception as e:
    print(f"Error: {e}")

property_list = []

for property in properties:
    try:
        property_data = {
            "company_name": property.find_element(By.XPATH, './/meta[@name="og:site_name"]').get_attribute("content") if property.find_elements(By.XPATH, './/meta[@name="og:site_name"]') else '',
            "property_title": property.find_element(By.XPATH, './/h2/a').text if property.find_elements(By.XPATH, './/h2/a') else '',
            "property_address": property.find_element(By.XPATH, './/span[@class="property-address"]').text if property.find_elements(By.XPATH, './/span[@class="property-address"]') else '',
            "postcode": property.find_element(By.XPATH, './/span[@class="property-postcode"]').text if property.find_elements(By.XPATH, './/span[@class="property-postcode"]') else '',
            "house_number": property.find_element(By.XPATH, './/span[@class="property-house-number"]').text if property.find_elements(By.XPATH, './/span[@class="property-house-number"]') else '',
            "state": property.find_element(By.XPATH, './/span[@class="property-state"]').text if property.find_elements(By.XPATH, './/span[@class="property-state"]') else '',
            "country": property.find_element(By.XPATH, './/span[@class="property-country"]').text if property.find_elements(By.XPATH, './/span[@class="property-country"]') else '',
            "property_type": property.find_element(By.XPATH, './/span[@class="property-type"]').text if property.find_elements(By.XPATH, './/span[@class="property-type"]') else '',
            "property_description": property.find_element(By.XPATH, './/div[@class="property-description"]').text if property.find_elements(By.XPATH, './/div[@class="property-description"]') else '',
            "property_map_coordinates": property.find_element(By.XPATH, './/meta[@name="geo.position"]').get_attribute("content") if property.find_elements(By.XPATH, './/meta[@name="geo.position"]') else '',
            "property_size": property.find_element(By.XPATH, './/span[@class="property-size"]').text if property.find_elements(By.XPATH, './/span[@class="property-size"]') else '',
            "date_added": property.find_element(By.XPATH, './/span[@class="property-date-added"]').text if property.find_elements(By.XPATH, './/span[@class="property-date-added"]') else '',
            "property_sale_price": property.find_element(By.XPATH, './/span[@class="property-sale-price"]').text if property.find_elements(By.XPATH, './/span[@class="property-sale-price"]') else '',
            "property_amenities": [amenity.text for amenity in property.find_elements(By.XPATH, './/div[@class="property-amenities"]/ul/li')],
            "property_images": [img.get_attribute("src") for img in property.find_elements(By.XPATH, './/div[@class="property-images"]//img')],
            "property_rent_details": property.find_element(By.XPATH, './/span[@class="property-rent-details"]').text if property.find_elements(By.XPATH, './/span[@class="property-rent-details"]') else '',
            "property_link": property.find_element(By.XPATH, './/a[@class="property-link"]').get_attribute("href") if property.find_elements(By.XPATH, './/a[@class="property-link"]') else '',
            "agent_name": property.find_element(By.XPATH, './/span[@class="agent-name"]').text if property.find_elements(By.XPATH, './/span[@class="agent-name"]') else '',
            "agent_name2": property.find_element(By.XPATH, './/span[@class="agent-name2"]').text if property.find_elements(By.XPATH, './/span[@class="agent-name2"]') else '',
            "agent_telephone": property.find_element(By.XPATH, './/span[@class="agent-telephone"]').text if property.find_elements(By.XPATH, './/span[@class="agent-telephone"]') else '',
            "agent_telephone2": property.find_element(By.XPATH, './/span[@class="agent-telephone2"]').text if property.find_elements(By.XPATH, './/span[@class="agent-telephone2"]') else '',
            "agent_email": property.find_element(By.XPATH, './/span[@class="agent-email"]').text if property.find_elements(By.XPATH, './/span[@class="agent-email"]') else '',
            "agent_email2": property.find_element(By.XPATH, './/span[@class="agent-email2"]').text if property.find_elements(By.XPATH, './/span[@class="agent-email2"]') else '',
            "agent_address": property.find_element(By.XPATH, './/span[@class="agent-address"]').text if property.find_elements(By.XPATH, './/span[@class="agent-address"]') else '',
            "property_sale_type": property.find_element(By.XPATH, './/span[@class="property-sale-type"]').text if property.find_elements(By.XPATH, './/span[@class="property-sale-type"]') else ''
        }
        print(property_data)  # Print each property data to debug
        property_list.append(property_data)
    except Exception as e:
        print(f"Error extracting property data: {e}")

# Save the data to a JSON file
output_file = 'property_data.json'  # Save in the current working directory
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(property_list, json_file, ensure_ascii=False, indent=4)

print(f"Data extracted and saved to {output_file}")

# Close the driver
driver.quit()


<html lang="en-US" dir="ltr"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error-text{background-image:url(